### ACID TRANSACTION

#### Data loaded

In [0]:
df_silver_events = spark.read.table("workspace.default.df_silver_events")
display(df_silver_events.limit(3))


date,event_year,user_id,product_id,event_type,category_code,category,brand,price
2019-11-16,2019,512563839,1004249,view,electronics.smartphone,electronics,apple,795
2019-11-16,2019,517136560,3600661,view,appliances.kitchen.washer,appliances,samsung,308
2019-11-16,2019,524088251,1004565,view,electronics.smartphone,electronics,huawei,166


#### Actions taken:- UPDATE, DELETE AND MERGE(UPSERT)

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(spark, "workspace.default.df_silver_events")

# UPDATE
deltaTable.update(
    condition="product_id = 11000101",
    set={"category_code": "'appliances.kitchen.toaster'"}
)

# DELETE
deltaTable.delete("user_id = 518497482 AND product_id = 11000101")

# MERGE (UPSERT)
newData = spark.createDataFrame([
    ("02-10-2019", 2019, 600123456, 20020020, "purchase", "electronics.laptop", "electronics", "lenovo", 850)
], ["date","event_year","user_id","product_id","event_type","category_code","category","brand","price"])

deltaTable.alias("target").merge(
    newData.alias("source"),
    "target.user_id = source.user_id AND target.product_id = source.product_id"
).whenMatchedUpdate(set={
    "event_type": "source.event_type",
    "price": "source.price"
}).whenNotMatchedInsert(values={
    "date": "source.date",
    "event_year": "source.event_year",
    "user_id": "source.user_id",
    "product_id": "source.product_id",
    "event_type": "source.event_type",
    "category_code": "source.category_code",
    "category": "source.category",
    "brand": "source.brand",
    "price": "source.price"
}).execute()



DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

#### Saved the actions in a Table

In [0]:
# Convert DeltaTable to DataFrame and save as a new table
updated_df = deltaTable.toDF()
updated_df.write.format("delta").mode("overwrite").saveAsTable("df_silver_events_1")

#### ACID transactions in Delta Lake ensure reliable updates, deletes, merges, and time travel, keeping the data consistent and trustworthy throughout the pipeline.